In [ ]:
import numpy as np
import os
import pandas
import json
from pathlib import Path
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [ ]:
rows = ["experiment_1", "experiment_2", "experiment_3"]
cols = ["experiment_name", "method_name", "inference_time (something proportional to time)"
       , "ground_truth_traj", "eval_traj", "l2_eval_error", "training_loss", "model_parameters"]

In [ ]:
loaded_df = pandas.read_pickle("../dataframe.pkl")

In [ ]:
fig, ax1 = plt.subplots(1)
fig.set_figwidth(16)
fig.set_figheight(8)
for index, row in loaded_df.iterrows():
    if row["system_name"] != "spring":
        continue
    if row["method_name"] == "srnn":
        style = "k"
        label = "srnn"
    elif row["method_name"] == "hnn":
        style = "b"
        label = "hnn"
    elif row["method_name"] == "mlp":
        style = "r"
        label = "mlp" 
    ax1.semilogy(row["inferred_trajectory_error"].mean(axis=0), style, label=label)
ax1.set_xlabel("Time step")
ax1.set_ylabel("Relative Error")
ax1.grid(b=True, which='both', axis='both')
#     plt.legend()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.set_figwidth(16)
fig.set_figheight(8)
for index, row in loaded_df.iterrows():
    if row["system_name"] != "spring":
        continue
    if row["method_name"] == "srnn":
        ax = ax1
    elif row["method_name"] == "hnn":
        ax = ax2
    elif row["method_name"] == "mlp":
        ax = ax3
    ax.semilogy(row["inferred_trajectory_error"].mean(axis=0), label="epochs_"+str(row["num_epochs"]))
ax1.grid(b=True, which='both', axis='both')
ax1.legend()
ax2.legend()
ax3.legend()


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.set_figwidth(16)
fig.set_figheight(8)
for index, row in loaded_df.iterrows():
    if row["system_name"] != "spring":
        continue
    if row["method_name"] == "srnn":
        ax = ax1
    elif row["method_name"] == "hnn":
        ax = ax2
    elif row["method_name"] == "mlp":
        ax = ax3
    ax.semilogy(row["inferred_trajectory_error"].mean(axis=0), label="trajectories_"+str(row["num_train_trajectories"]))
ax1.grid(b=True, which='both', axis='both')
ax1.legend()
ax2.legend()
ax3.legend()



In [ ]:
loaded_df.columns

In [ ]:
loaded_df.drop(['ground_truth_trajectory', 'inferred_trajectory', 'inferred_trajectory_error'], axis=1)